In [ ]:
%matplotlib notebook
%pylab
# %matplotlib inline
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format='retina'

from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from scipy.stats import poisson
from scipy.optimize import curve_fit
import numpy as np
import os
import pickle as pkl
from cmcrameri import cm
import pandas as pd
import h5py

def folder_structure(name, node):
    if isinstance(node, h5py.Group):
        print(node.name)
    elif isinstance(node, h5py.Dataset):
        if (node.dtype == 'object') :
            print (node.name, 'contains numeric data')
        else:
            print('\t', node.name, node.shape)

            
from BDplotTools import plot_dynamics, combine_Gammas
from scipy.constants import Boltzmann,pi,e
import matplotlib.patheffects as pe

def combine_Gammas(dataSets,result_ids,q_ind, ax = None):
    '''Plot gamma vs dose curves from several series in one figure    

    Parameters
    ----------
    dataSets: data frame
        data frame with name and number of each data set
    result_ids: list of tuples
        list of tuples with temp, meas_id, series, absorber, number of exponents in fit
    q_ind: int
        number of q ROI
    ax: axis
        axis for plotting


    Returns
    -------

    
    '''    

    colors = plt.cm.turbo_r(np.linspace(1,0,10))
    absorber_index = {'24':0,'18':1,'16':2,'12':3,'8':4,'6':5,'4':6,'2':7,'1':8,'0':9}
    flux_dict={'24':3.36e7,'18':2.18e8,'16':4.08e8,'12':1.42e9,'8':4.95e9,'6':9.23e9,'4':1.72e10,'2':3.21e10,'1':4.39e10,'0':6e10,}
    marker_size=6
    mec = 'gray'
    mew = 0.2
    for temp, meas_no, ser, absorber, no_exp in result_ids:
        with h5py.File('hdf5Results/' + dataSets['measurement'][meas_no]+'.hdf5','r') as f:
            if(no_exp==1):
                Gamma = f['/scan_{:.0f}/q_{:.0f}/horizontal_cuts/1_exp_fit/fit_parameter_results'.format(ser,q_ind)][:,1]
                Dose = f['/scan_{:.0f}/q_{:.0f}/horizontal_cuts/1_exp_fit/dose_kGy'.format(ser,q_ind)][:]
                G_err = f['/scan_{:.0f}/q_{:.0f}/horizontal_cuts/1_exp_fit/fit_parameter_errors'.format(ser,q_ind)][:,1]
                
                ax.errorbar(x=Dose,
                    y=Gamma,
                    yerr=G_err,
                    color = colors[absorber_index[str(absorber)]],
                    marker='s',
                    markersize = marker_size,
                        mec=mec,mew=mew,
                    linestyle='',fillstyle='full')
            else:
                Gamma = f['/scan_{:.0f}/q_{:.0f}/horizontal_cuts/2_exp_fit/fit_parameter_results'.format(ser,q_ind)][:,2]
                Dose = f['/scan_{:.0f}/q_{:.0f}/horizontal_cuts/2_exp_fit/dose_kGy'.format(ser,q_ind)][:]
                G_err = f['/scan_{:.0f}/q_{:.0f}/horizontal_cuts/2_exp_fit/fit_parameter_error'.format(ser,q_ind)][:,2]
                Gamma2 = f['/scan_{:.0f}/q_{:.0f}/horizontal_cuts/2_exp_fit/fit_parameter_results'.format(ser,q_ind)][:,4]
                G_err2 = f['/scan_{:.0f}/q_{:.0f}/horizontal_cuts/2_exp_fit/fit_parameter_error'.format(ser,q_ind)][:,4]

                ax.errorbar(x=Dose,
                                y=Gamma2,
                                yerr=G_err2,
                                color = colors[absorber_index[str(absorber)]],
                                marker='s',
                                markersize = marker_size,
                                    mec=mec,mew=mew,
                                linestyle='',fillstyle='full')
                
            
    return 

def combine_KWWs(dataSets,result_ids,q_ind, ax = None):
    '''Plot KWW vs dose curves from several series in one figure    

    Parameters
    ----------
    dataSets: data frame
        data frame with name and number of each data set
    result_ids: list of tuples
        list of tuples with temp, meas_id, series, absorber, number of exponents in fit
    q_ind: int
        number of q ROI
    ax: axis
        axis for plotting


    Returns
    -------

    
    '''
    colors = plt.cm.turbo_r(np.linspace(1,0,10))
    absorber_index = {'24':0,'18':1,'16':2,'12':3,'8':4,'6':5,'4':6,'2':7,'1':8,'0':9}
    flux_dict={'24':3.36e7,'18':2.18e8,'16':4.08e8,'12':1.42e9,'8':4.95e9,'6':9.23e9,'4':1.72e10,'2':3.21e10,'1':4.39e10,'0':6e10,}
    marker_size=3
    mec = 'gray'
    mew = 0.2
    for temp, meas_no, ser, absorber, no_exp in result_ids:
        with h5py.File('hdf5Results/' + dataSets['measurement'][meas_no]+'.hdf5','r') as f:
            if(no_exp==1):
                Dose = f['/scan_{:.0f}/q_{:.0f}/horizontal_cuts/1_exp_fit/dose_kGy'.format(ser,q_ind)][:]
                K_err = f['/scan_{:.0f}/q_{:.0f}/horizontal_cuts/1_exp_fit/fit_parameter_errors'.format(ser,q_ind)][:,2]
                KWW = f['/scan_{:.0f}/q_{:.0f}/horizontal_cuts/1_exp_fit/fit_parameter_results'.format(ser,q_ind)][:,2]
            else:
                KWW = f['/scan_{:.0f}/q_{:.0f}/horizontal_cuts/2_exp_fit/fit_parameter_results'.format(ser,q_ind)][:,3]
                Dose = f['/scan_{:.0f}/q_{:.0f}/horizontal_cuts/2_exp_fit/dose_kGy'.format(ser,q_ind)][:]
                K_err = f['/scan_{:.0f}/q_{:.0f}/horizontal_cuts/2_exp_fit/fit_parameter_error'.format(ser,q_ind)][:,3]

            ax.errorbar(x=Dose,
                    y=KWW,
                    yerr=K_err,
                    color = colors[absorber_index[str(absorber)]],
                    marker='o',
                    markersize = marker_size,
                    linestyle='',fillstyle='full')
                
            
    return 

def weighted_average(vals,errs): # calculate weighted average 
    eps = 10E-14
    weights = np.divide(1,np.power(errs+eps,2))
    mean = np.sum(np.multiply(weights,vals))/np.sum(weights)
    err = 1/np.sqrt(np.sum(weights))
    return mean, err

def Dosis(t, ab):
    ''' formula for dose calculation '''
    Phi_c = 6e10*(0.729**ab)
    E_c =8540*e  #J/ph photon energy
    d_s =0.0015 #m sample thickness
    A = 0.72# absorption in 1mm of water
    Phi = 1 # volume fraction
    z_sq = 100e-06*100e-06 #m^2 beam area

    rho = 1000 #kg/m^3 sample mass density
    return(np.multiply(t,Phi_c*E_c*A*Phi/(z_sq*d_s*rho))/1000) #return Dose in kGy

def dDosedTau(ab):
    ''' First derivative of dosis with respect to exposure time'''
    Phi_c = 6e10*(0.729**ab)
    E_c =8540*e  #J/ph photon energy
    d_s =0.0015 #m sample thickness
    A = 0.72# absorption in 1mm of water
    Phi = 1 # volume fraction
    z_sq = 100e-06*100e-06 #m^2 beam area

    rho = 1000 #kg/m^3 sample mass density
    return(np.multiply(1,Phi_c*E_c*A*Phi/(z_sq*d_s*rho))/1000) #return Dose in kGy)

def GetSetup(setupfile = './setupfiles/example_setup.pkl'):
    setupFile = open(setupfile,'rb')
    setupParams = pkl.load(setupFile)
    setupFile.close()
    
    qValues = setupParams.qv #all properties are listed under https://github.com/reiserm/Xana/blob/master/Xana/Setup.py
    maskFile = setupParams.maskfile
    beamCenter = setupParams.center
    qRoi = setupParams.qroi
    
    return([qValues,maskFile,qRoi,beamCenter])

In [ ]:
plt.style.use('paper_style.mplstyle')

In [ ]:
dataSets = pd.read_csv('measurement_set.csv', delimiter=';',header=0,)
logbook = pd.read_csv('egg_logbook.csv', delimiter=';',header=0,)

sublog = logbook.loc[logbook.meas_id==4]
print(sublog)
print(sublog.iloc[0]['measurement'])

In [ ]:
def g2_model_2exp(t, beta, beta_0, Gamma1, kww1, Gamma2, kww2,off):
                g2 = 1 + off+ beta * \
                    (beta_0 * np.exp(-2 * (Gamma1 * t) ** (kww1)) + (1-beta_0) * \
                    np.exp(-2 * (Gamma2 * t) ** (kww2)))
                return g2
def g2_model(t, beta, Gamma, kww, off):
    g2 = np.add(1 + off , beta * np.exp(-2 * (Gamma * t) ** kww))
    return g2

def add_colorbar(ax, vec, label=None,cmap = mpl.cm.jet,fontsize=12):
    vec = np.array(vec)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='5%', pad=0.05)

    
    norm = mpl.colors.Normalize(vmin=vec.min(), vmax=vec.max())

    cb1 = mpl.colorbar.ColorbarBase(cax, cmap=cmap,norm=norm,orientation='vertical')
    cb1.set_label(label,fontsize=fontsize)
    
def Dosis(t, ab):
    Phi_c = 6e10*(0.729**ab)
    E_c =8540*e  #J/ph photon energy
    d_s =0.0015 #m sample thickness
    A = 0.72# absorption in 1mm of water
    Phi = 1 # volume fraction
    z_sq = 100e-06*100e-06 #m^2 beam area

    rho = 1000 #kg/m^3 sample mass density
    return(np.multiply(t,Phi_c*E_c*A*Phi/(z_sq*d_s*rho))/1000) #return Dose in kGy

# Figure 3

In [ ]:
fig = plt.figure(figsize=(7,np.divide(7,3)))
ax0=plt.axes([0.04,0.15,np.divide(0.8,3),0.65])
ax1=plt.axes([0.38,0.15,np.divide(0.8,3),0.65])
ax2=plt.axes([0.73,0.15,np.divide(0.75,3),0.65])

pos0 = ax0.get_position() # get the original position 
ax0.text(-0.25,1.2,'a',fontsize=18,fontweight='bold',transform = ax0.transAxes,color='black')

pos1 = ax1.get_position() # get the original position 
ax1a = fig.add_axes([0.57, 0.53, 0.07, pos1.height*0.3])
ax1a.set_ylabel('$\mathrm{\Gamma\:(s^{-1})}$',fontsize=6)
ax1a.set_xlabel('$\mathcal{D}\:\mathrm{(kGy)}$',fontsize=6)
#ax1a.ticklabel_format(axis='y',style='sci',scilimits=(0,0),useMathText=True)
ax1a.tick_params(axis='both', which='major', labelsize=6)
#ax1a.set_yscale('log')
# ax1a.set_yticks([0.05,0.1,0.15,0.2,0.25])
# ax1a.set_yticklabels(['0.05','','0.15','','0.25'],fontsize=8)
ax1a.set_xticks([0,5,10])
ax1a.set_xticklabels(['0','5','10'],fontsize=6)
ax1.text(-0.25,1.2,'b',fontsize=18,transform = ax1.transAxes,color='black',fontweight='bold')
ax1.set_xlim([0.2,250])

pos2 = ax2.get_position() # get the original position 
ax2a = fig.add_axes([0.905, 0.53, 0.07, pos2.height*0.3])
ax2b = fig.add_axes([0.765, 0.27, 0.07, pos2.height*0.3])
ax2a.set_ylabel('$\mathrm{\Gamma\:(s^{-1})}$',fontsize=6)
ax2a.set_xlabel('$q\:\mathrm{(\mu m^{-1})}$',fontsize=6)
#ax2a.ticklabel_format(axis='y',style='sci',scilimits=(0,0),useMathText=True)
#ax2a.set_yticklabels(['','2.5',],fontsize=10)
ax2a.set_xticks([0,0.005,0.01,0.015])
ax2a.set_xticklabels(['0','','10',''],fontsize=8)

ax2b.set_ylabel('  $k$',labelpad=-6,fontsize=6)
ax2b.set_xlabel('$q\:\mathrm{(\mu m^{-1})}$',fontsize=6)
ax2b.set_xticks([0.006,0.008,0.01,0.012,0.014])
ax2b.set_xticklabels(['6','','10','','14'],fontsize=6)
ax2b.set_yticks([1.5,2.0])
ax2b.set_yticklabels(['1.5','2'],fontsize=6)

ax2a.tick_params(axis='both', which='major', labelsize=6)
ax2b.tick_params(axis='both', which='major', labelsize=6)
ax2.text(-0.25,1.2,'c',fontsize=18,transform = ax2.transAxes,color='black',fontweight='bold')
ax2a.yaxis.offsetText.set_fontsize(6)
# ax2a.set_ylim([0,0.02])
# ax2a.set_xlim([0,0.068])
ax2.set_xlim([0.2,250])
ax2.set_ylim([0.99,1.075])

q_ind=0
scan=4

with h5py.File('hdf5Results/' + sublog.iloc[0]['measurement']+'.hdf5','r') as f:
    ####
    ttc = f[f'/scan_{scan}/q_{q_ind}/ttc/ttc'][()]
    tt = f[f'/scan_{scan}/q_{q_ind}/ttc/t'][()]
    im = ax0.imshow(ttc,origin='lower',extent=([tt[0],tt[-1],tt[0],tt[-1]]),vmin=1,vmax=1.08)
    divider = make_axes_locatable(ax0)
    cax = divider.append_axes('top', size='5%', pad=0.05)
    norm = mpl.colors.Normalize(vmin=1, vmax=1.12)
    cb1 = mpl.colorbar.ColorbarBase(cax, cmap=plt.cm.viridis,norm=norm,orientation='horizontal')
    cb1.set_label('$\mathrm{c^{(2)}(t_1,t_2)}$',labelpad=5)
    cax.xaxis.set_ticks_position("top")
    cax.xaxis.set_label_position("top")
    
    ########
    g2_tcut = f[f'/scan_{scan}/q_{q_ind}/horizontal_cuts/1_exp_fit/g2'][()]
    tau_tcut = f[f'/scan_{scan}/q_{q_ind}/horizontal_cuts/1_exp_fit/tau'][()]
    tcut = f[f'/scan_{scan}/q_{q_ind}/horizontal_cuts/1_exp_fit/t_cut'][()]
    popt = f[f'/scan_{scan}/q_{q_ind}/horizontal_cuts/1_exp_fit/fit_parameter_results'][()]
    pcov = f[f'/scan_{scan}/q_{q_ind}/horizontal_cuts/1_exp_fit/fit_parameter_errors'][()]
    colors = plt.cm.RdYlBu_r(np.linspace(0,1,20))
    for itcut in range(0,20):
        ax1.plot(tau_tcut[itcut,:],g2_tcut[itcut,:],color=colors[itcut],marker='o',linestyle='',markersize=2)
        tau_range_plot = np.logspace(-2.4,2,50)
        line2 = ax1.plot(tau_range_plot, g2_model(tau_range_plot,*popt[itcut,:]), c=colors[itcut], ls='-', ) 
        ax1a.errorbar(x=Dosis(tcut[itcut],ab=12),y=popt[itcut,1],yerr=pcov[itcut,1],color=colors[itcut],marker='o',markersize=4)
    #add_colorbar(ax1,tcut,'$\mathrm{t_{cut}(s)}$',cmap=plt.cm.Spectral_r)
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes('top', size='5%', pad=0.05)
    norm = mpl.colors.Normalize(vmin=Dosis(tcut.min(),ab=12), vmax=Dosis(tcut.max(),ab=12))
    cb1 = mpl.colorbar.ColorbarBase(cax, cmap=plt.cm.RdYlBu_r,norm=norm,orientation='horizontal')
    cb1.set_label('$\mathcal{D}\:\mathrm{(kGy)}$',labelpad=5)
    cax.xaxis.set_ticks_position("top")
    cax.xaxis.set_label_position("top")
    cax.xaxis.set_ticks([0,2,4,6,8,10])
    cax.xaxis.set_ticklabels([0,2,4,6,8,10])
    

    
    #######
    g2_q = f[f'/scan_{scan}/constant_dose/1kGy/g2'][()]
    tau_q = f[f'/scan_{scan}/constant_dose/1kGy/tau'][()]
    q_labels = f[f'/scan_{scan}/constant_dose/1kGy/q_nm'][()]
    poptq = f[f'/scan_{scan}/constant_dose/1kGy/fit_parameter_results'][()]
    perrq = f[f'/scan_{scan}/constant_dose/1kGy/fit_parameter_errors'][()]
    v= f[f'/scan_{scan}/constant_dose/1kGy/v_nm_s'][0]
    verr= f[f'/scan_{scan}/constant_dose/1kGy/v_err'][0]

    colorsq = cm.turku(np.linspace(0,1,5))
    for qcut in range(5):
        #print(poptq[qcut,0])
        ax2.plot(tau_q,np.subtract(g2_q[qcut,:],1+poptq[qcut,-1])+1,color=colorsq[qcut],marker='o',
                 linestyle='',markersize=3,label=f'{q_labels[qcut]:.3f}',mec='black',mew=0.2)
        tau_range_plot = np.logspace(-2.4,2.2,50)
        line3 = ax2.plot(tau_range_plot, np.subtract(g2_model(tau_range_plot,*poptq[qcut,:]),1+poptq[qcut,-1])+1, 
                         c=colorsq[qcut], ls='-',zorder=-1 )
        ax2a.errorbar(x=q_labels[qcut],y=poptq[qcut,1],yerr=perrq[qcut,1],color=colorsq[qcut],marker='o',markersize=4,mec='black',mew=0.2)
        ax2b.errorbar(x=q_labels[qcut],y=poptq[qcut,2],yerr=perrq[qcut,2],color=colorsq[qcut],marker='o',markersize=4,mec='black',mew=0.2)
    def line(x,a):
        return x*a
    
    pars, cov = curve_fit(line,q_labels,poptq[:,1])
    ax2a.plot(np.arange(0,0.02,0.005),pars*np.arange(0,0.02,0.005),color='black',zorder=-10)
        
        
    divider2 = make_axes_locatable(ax2)
    cax2 = divider2.append_axes('top', size='5%', pad=0.05)
    cmap2 = ListedColormap([colorsq[i] for i in range(5)])
    norm2 = mpl.colors.Normalize(vmin=0,vmax=5)
    cb2 = mpl.colorbar.ColorbarBase(cax2, cmap=cmap2,norm=norm2,orientation='horizontal')
    cb2.set_ticks([0.5,2.5,4.5])
    cb2.set_ticklabels([q_labels[0],q_labels[2],q_labels[4]])
    cb2.set_label('$q\:\mathrm{(nm^{-1})}$',labelpad=5)
    cax2.xaxis.set_ticks_position("top")
    cax2.xaxis.set_label_position("top")

for j in range(2,20,3):
    ax0.arrow(tcut[j],tcut[j],190-tcut[j],0,head_width=4,color=colors[j],length_includes_head=True)

    #########
#     gamma = poptq[:,1]
#     for qcut in range(10):
#         ax[1,1].errorbar(x=q_labels[qcut],y=gamma[qcut],yerr=perrq[qcut,1],linestyle='',marker='o',color=colorsq[qcut],capsize=2,markersize=8)
        
#     pars1, cov1 = np.polyfit(x=q_labels,y=gamma,deg=1,cov=True)
#     ax[1,1].plot(q_labels,pars1[0]*q_labels+pars1[1],color='black', linestyle=(0,(5,5)))
#     ax[1,1].annotate(text='v = {:.0f}'.format(v)+'$\pm$'+'{:.1f} nm/s'.format(verr) ,xy=(0.005,.3))#+' nm $s^{-1} ^\circ C ^{-1}$'
    
for a in [ax1,ax2]:
    a.set_xscale('log')


    
ax0.set_ylabel(r'$t_2\:\mathrm{(s)}$')
ax0.set_xlabel(r'$t_1\:\mathrm{(s)}$')
ax1.set_ylabel(r'$\mathrm{g^{(2)}(\tau)}$')
ax1.set_xlabel(r'$\tau\:\mathrm{(s)}$')
ax2.set_ylabel(r'$\mathrm{g^{(2)}(\tau)}$')
ax2.set_xlabel(r'$\tau\:\mathrm{(s)}$')
# # ax[1,1].set_ylabel(r'$\mathrm{\Gamma(q)} (s^{-1})$')
# # ax[1,1].set_xlabel(r'$\mathrm{q(nm^{-1})}$')
# plt.tight_layout()

# fig.show()
plt.savefig('Figure3.pdf',dpi=1200,format='pdf')

# Figure 4

In [ ]:
fig, ax = plt.subplots(ncols=2,nrows=2,sharex=True,sharey=True,figsize=(7,4),squeeze=True)
plt.subplots_adjust(right=0.77,wspace=0,hspace=0,left=0.1,bottom=0.1,top=0.95)
ax0, ax1, ax2, ax3 = ax[0,0],ax[0,1],ax[1,0],ax[1,1]

pos0 = ax0.get_position() # get the original position 
ax0a = fig.add_axes([pos0.x0+0.6*pos0.width, pos0.y0+0.18*pos0.height, 0.3*pos0.width, 0.3*pos0.height])
pos1 = ax1.get_position() # get the original position 
ax1a = fig.add_axes([pos1.x0+0.6*pos1.width, pos1.y0+0.18*pos1.height, 0.3*pos1.width, 0.3*pos1.height])
pos2 = ax2.get_position() # get the original position 
ax2a = fig.add_axes([pos2.x0+0.22*pos2.width, pos2.y0+0.67*pos2.height, 0.3*pos0.width, 0.3*pos0.height])
pos3 = ax3.get_position() # get the original position 
ax3a = fig.add_axes([pos3.x0+0.22*pos3.width, pos3.y0+0.67*pos3.height, 0.3*pos0.width, 0.3*pos0.height])

for a in [ax0a,ax1a,ax2a,ax3a]:
    a.set_xscale('log')
    a.set_xticks([0.1,1,10,100])
    a.set_xticklabels(['$10^{-1}$','$10^{0}$','$10^{1}$','$10^2$'],fontsize=6)
    a.set_yticks([0,1,2,3])
    a.set_yticklabels([0,1,2,3],fontsize=6)
    a.set_xlabel('$\mathcal{D}$(kGy)',fontsize=8)
    a.set_ylabel('$\mathcal{k}$',fontsize=8)
    a.set_ylim([0,3.2])
    a.set_xlim([0.05,700])

colors = plt.cm.turbo_r(np.linspace(1,0,10))
ax_col_flux = fig.add_axes([0.88,0.1,0.02,0.85])
ax_col_doserate = ax_col_flux.twinx()

cmap = ListedColormap([colors[i] for i in range(10)])
norm = mpl.colors.Normalize(vmin=0,vmax=10)
cb = mpl.colorbar.ColorbarBase(ax_col_doserate, cmap=cmap,norm=norm,orientation='vertical')
cb.set_ticks(np.arange(0.5,10.5,1))
cb.set_ticklabels([0.002,0.01,0.03,0.09,0.3,0.6,1,2,3,4])
cb.set_label('Dose rate $\mathscr{D}\:\mathrm{(kGy\:s^{-1})}$',fontsize=12,labelpad=0)

ax_col_flux.set_ylim([0,10])
ax_col_flux.set_yticks(np.arange(0.5,10.5,1))
ax_col_flux.set_yticklabels([r'3$\times 10^{-3}$',r'2$\times 10^{-2}$',r'4$\times 10^{-2}$',r'1$\times 10^{-1}$',
                             r'5$\times 10^{-1}$',r'9$\times 10^{-1}$',r'2$\times 10^{0}$',r'3$\times 10^{0}$',
                             r'4$\times 10^{0}$',r'6$\times 10^{0}$'])

ax_col_flux.set_ylabel(r'Photon fluence $\Phi$ ($\mathrm{ph}\:\mathrm{s}^{-1}\:\mathrm{nm}^{-2}$)')
for a,label,temp_label,q_label in zip([ax0, ax1, ax2, ax3],
                              ['a','b','c','d'],
                              ['$\mathrm{T_{prep}}=63\degree$C','$\mathrm{T_{prep}}=70\degree$C','$\mathrm{T_{prep}}=73\degree$C','$\mathrm{T_{prep}}=78\degree$C'],
                              ['q=0.006$\mathrm{\:nm^{-1}}$','q=0.006$\mathrm{\:nm^{-1}}$',
                               'q=0.02$\mathrm{\:nm^{-1}}$','q=0.02$\mathrm{\:nm^{-1}}$']):
    a.set_yscale('log')
    a.set_xscale('log')
    a.set_xlim([0.05,700])
    a.set_ylim([0.0002,20])
    a.grid('on',alpha=0.3)
    a.text(0.03,0.86,label,fontsize=18,transform = a.transAxes,color='black',fontweight='bold')
    a.text(0.53,0.9,temp_label,fontsize=12,transform = a.transAxes,color='black')
    
ax0.text(0.02,0.75,'q=0.006$\mathrm{\:nm^{-1}}$',fontsize=8,transform = ax0.transAxes,color='black')
ax1.text(0.02,0.02,'q=0.006$\mathrm{\:nm^{-1}}$',fontsize=8,transform = ax1.transAxes,color='black')
ax2.text(0.02,0.02,'q=0.02$\mathrm{\:nm^{-1}}$',fontsize=8,transform = ax2.transAxes,color='black')
ax3.text(0.5,0.02,'q=0.02$\mathrm{\:nm^{-1}}$',fontsize=8,transform = ax3.transAxes,color='black')

ax0.set_ylabel('$\mathrm{\Gamma\:(s^{-1})}$') 
ax2.set_ylabel('$\mathrm{\Gamma\:(s^{-1})}$')
ax2.set_xlabel('Accumulated dose $\mathcal{D}\:\mathrm{(kGy)}$')
ax3.set_xlabel('Accumulated dose $\mathcal{D}\:\mathrm{(kGy)}$')

###########63C###############
sublog = logbook.loc[logbook.temp==63]
result_ids = []
use_entries = [0,1,2,3,5,6,7,8,9,10,11,12,13,14,15]
exponentiality = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
for ie,(ue,ex) in enumerate(zip(use_entries,exponentiality)):
    res_id = sublog.iloc[ue]['temp'],sublog.iloc[ue]['meas_id'],sublog.iloc[ue]['ser'],sublog.iloc[ue]['ab'],ex
    result_ids.append(res_id)
    
q_ind=0

with h5py.File('hdf5Results/' + sublog.iloc[0]['measurement']+'.hdf5','r') as f:
    combine_Gammas(dataSets,result_ids,q_ind,ax=ax0)
    combine_KWWs(dataSets,result_ids,q_ind,ax=ax0a)
    
###########70C###############
sublog = logbook.loc[logbook.temp==70]
result_ids = []
use_entries = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
exponentiality = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
for ie,(ue,ex) in enumerate(zip(use_entries,exponentiality)):
    res_id = sublog.iloc[ue]['temp'],sublog.iloc[ue]['meas_id'],sublog.iloc[ue]['ser'],sublog.iloc[ue]['ab'],ex
    result_ids.append(res_id)
    
q_ind=0

with h5py.File('hdf5Results/' + sublog.iloc[0]['measurement']+'.hdf5','r') as f:
    combine_Gammas(dataSets,result_ids,q_ind,ax=ax1)
    combine_KWWs(dataSets,result_ids,q_ind,ax=ax1a)
    
    
###########73C###############
sublog = logbook.loc[logbook.temp==73]
result_ids = []
use_entries = [0,1,2,3,4,6,8,10,12,14]
exponentiality = [1,1,1,1,1,1,1,2,2,2]
for ie,(ue,ex) in enumerate(zip(use_entries,exponentiality)):
    res_id = sublog.iloc[ue]['temp'],sublog.iloc[ue]['meas_id'],sublog.iloc[ue]['ser'],sublog.iloc[ue]['ab'],ex
    result_ids.append(res_id)
    
q_ind=5

with h5py.File('hdf5Results/' + sublog.iloc[0]['measurement']+'.hdf5','r') as f:
    combine_Gammas(dataSets,result_ids,q_ind,ax=ax2)
    combine_KWWs(dataSets,result_ids,q_ind,ax=ax2a)
    
###########78C###############
sublog = logbook.loc[logbook.temp==78]
result_ids = []
use_entries =    [2,3,4,8,10,12,14]
exponentiality = [1,1,1,1, 2, 2, 2]
for ie,(ue,ex) in enumerate(zip(use_entries,exponentiality)):
    res_id = sublog.iloc[ue]['temp'],sublog.iloc[ue]['meas_id'],sublog.iloc[ue]['ser'],sublog.iloc[ue]['ab'],ex
    result_ids.append(res_id)
    
q_ind=5

with h5py.File('hdf5Results/' + sublog.iloc[0]['measurement']+'.hdf5','r') as f:
    combine_Gammas(dataSets,result_ids,q_ind,ax=ax3)
    combine_KWWs(dataSets,result_ids,q_ind,ax=ax3a)
    
plt.savefig('Figure4.pdf',dpi=1200,format='pdf')

# Figure 5

In [ ]:
fig = plt.figure(figsize=(3.4,2.5))
gs2 = GridSpec(1, 1, left=0.15, bottom=0.15,top=0.95, right=0.75)
ax2 = fig.add_subplot(gs2[:, :])

ax2.set_xlabel('$t_1\mathrm{ \:(s)}$')
ax2.set_ylabel('$t_2\mathrm{ \:(s)}$')

ax2.set_yticks([500,1000,1500,2000,2500,3000,])
ax2.set_yticklabels(['500','','1500','','2500','',])
ax2.set_xticks([500,1000,1500,2000,2500,3000,])
ax2.set_xticklabels(['500','','1500','','2500','',])

ax_col_ttc = fig.add_axes([0.77,0.15,0.03,0.8])
cmap = plt.cm.viridis
norm = mpl.colors.Normalize(vmin=1,vmax=1.1)
cb = mpl.colorbar.ColorbarBase(ax_col_ttc, cmap=cmap,norm=norm,orientation='vertical')
cb.set_label(r'$c^{(2)}(t_1,t_2)$')

ax2.set_xlim([0,3450])
ax2.set_ylim([0,3450])
with h5py.File('hdf5Results/' + dataSets['measurement'][21]+'.hdf5','r+') as f:
    for scan, offset,t_delay, frames in zip([5,6,7,8,9],[0,1011,1222,2236,2450],[0.5,0.2,0.5,0.2,0.5],[2000,1000,2000,1000,2000]):
        ttc = f['/scan_{:.0f}/q_{:.0f}/ttc/ttc'.format(scan,6)][:,:]
        length = frames*t_delay
        extent=(offset,offset+length,offset,offset+length)
        
        ax2.imshow(ttc,cmap='viridis', vmin=1, vmax=1.15,origin='lower', extent=extent)
        
#ax2.text(1,0.8,r'$c^{(2)}(t_1,t_2)$',fontsize=8,transform = ax2.transAxes)
ax2.text(0.02,0.92,r'$\mathrm{T_{prep}}\:=\:80\:\degree$C',fontsize=10,transform = ax2.transAxes)
ax2.text(0.02,0.83,'$q=0.03\mathrm{\:nm^{-1}}$',fontsize=10,transform = ax2.transAxes)
#ax2.text(0.03,0.9,'c',fontweight='bold',fontsize=12,transform = ax2.transAxes,color='black')

#ax2.text(1700,200,'$0.33\:\mathrm{(kGy\:s^{-1})}$',fontsize=8,color='black',path_effects=[pe.Stroke(linewidth=1, foreground=colors[4],alpha=1)])
ax2.text(1700,300,'$0.002\:\mathrm{kGy\:s^{-1}}$',fontsize=8,color='black')
ax2.text(100,1300,'$2\:\mathrm{kGy\:s^{-1}}$',fontsize=8,color='black')
ax2.text(1300,2500,'$4\:\mathrm{kGy\:s^{-1}}$',fontsize=8,color='black')
ax2.arrow(1600,390,-500,0,head_width=70,color='gray',length_includes_head=True)
ax2.arrow(2100,600,-0,500,head_width=70,color='gray',length_includes_head=True)
ax2.arrow(2700,600,-0,1500,width=2,head_width=70,color='gray',length_includes_head=True)

ax2.arrow(700,1250,200,-100,head_width=30,color='gray',length_includes_head=True)
ax2.arrow(1900,2450,200,-100,head_width=30,color='gray',length_includes_head=True)

plt.savefig('Figure5.pdf',dpi=1200,format='pdf')

# Figure 6

In [ ]:
fig, ax = plt.subplots(nrows=2,ncols=2,figsize=(7,5))
plt.subplots_adjust(left=0.15,wspace=0.25,hspace=0,right=0.99,top=0.99,bottom=0.1)
ax0,ax1 = ax[0,0], ax[1,0]
ax2,ax3 = ax[0,1], ax[1,1]
q_values,maskfile,_,beam_center = GetSetup('setupfiles/USAXS_e4m_12q.pkl')

entries = [
(63,4,0,[1,2,3,4,6],[24,18,16,12,8]), 
(65,5,0,[1,2,3,4,6,8,9,11,13],[24,18,16,12,8,6,4,2,1]),
(68,6,0,[1,2,3,4,6,7,9,11],[24,18,16,12,8,6,4,2]),
(70,7,0,[1,2,3,4,6,7,9,11,13],[24,18,16,12,8,6,4,2,1]),
(73,8,0,[1,2,3,4,5,7,9,11,13,15],[24,18,16,12,8,6,4,2,1,0]),
(75,9,5,[3,4,5,7,9,11,13,15],[16,12,8,6,4,2,1,0]),
(78,10,5,[3,4,5,9,11,13,15],[16,12,8,4,2,1,0]),
(80,11,5,[3,4,5,7,9,11,13,15],[16,12,8,6,4,2,1,0]),
(85,12,5,[2,4,8,10],[18,8,2,0]),
]

def line_fit(x,Gamma0,alpha):
    return Gamma0+alpha*x

colors = plt.cm.seismic(np.linspace(0.15,1,9))

sample_color_dict = {'63':colors[0],'65':colors[1],'68':colors[2],
                     '70':colors[3],'73':colors[4],'75':colors[5],'78':colors[6],'80':colors[7],'85':colors[8]}

marker_dict = {'63':'o','65':'v','68':'^',
                     '70':'<','73':'>','75':'s','78':'D','80':'P','85':'p'}


flux_dict={'24':0.00336,'18':0.0218,'16':0.0408,'12':0.142,'8':0.495,'6':0.923,'4':1.72,'2':3.21,'1':4.39,'0':6,} #ph/s/nm^2
absorber_index = {'24':0,'18':1,'16':2,'12':3,'8':4,'6':5,'4':6,'2':7,'1':8,'0':9}


left, bottom, width, height = [0.41, 0.63, 0.1, 0.07]
ax0a = fig.add_axes([left, bottom, width, height])
left, bottom, width, height = [0.41, 0.18, 0.1, 0.07]
ax1a = fig.add_axes([left, bottom, width, height])

for a in [ax0a,ax1a]:
    a.set_ylabel(r'$\alpha\:( \mathrm{nm^3\:ph^{-1}}$)',fontsize=8)
    a.set_xlabel(r'$\mathrm{v_0\:(nm\: s^{-1}}$)',fontsize=8)
    a.tick_params(axis='both', which='major', labelsize=8)
    a.set_xscale('log')
    a.set_yscale('log')
    a.grid('on')

    

ax1.set_ylim([1e-3,8])
ax0.set_ylim([0.08,1700])
ax2.set_ylim([0.07,3])
ax3.set_ylim([0.05,2000])
ax0a.set_xlim([0.1,40])
ax0a.set_ylim([20,500])
ax1a.set_ylim([0.02,2])

v0s,dv0s = np.zeros((9,)),np.zeros((9,))
phid_soft = []
phid_strong=[]

for ie, (temp, meas_id,q_ind, ser_list, abs_list) in enumerate(entries):
    if temp<73:
        ax=ax0
        ax_i = ax0a
        ax_d = ax2
        q_temp = 0.006
    else:
        ax=ax1
        ax_i=ax1a
        ax_d=ax3
        q_temp = 0.02
        
    
    print('temp ' + str(temp))
    v1kGy_nms = np.zeros([len(ser_list)])
    dv1kGy_nms = np.zeros([len(ser_list)])
    fluxes = [flux_dict[str(absorber)] for absorber in abs_list]
    for im, scan in enumerate(ser_list):
        with h5py.File('hdf5Results/' + dataSets['measurement'][meas_id]+'.hdf5','r') as f:
            v1kGy_nms[im] = f[f'/scan_{scan}/constant_dose/1kGy/v_nm_s'][()]
            dv1kGy_nms[im] = f[f'/scan_{scan}/constant_dose/1kGy/v_err'][()]

    
    decay_doses = np.zeros([len(ser_list)])
    ddecay_doses = np.zeros([len(ser_list)])
    for im, (scan, ab) in enumerate(zip(ser_list,abs_list)):
        with h5py.File('hdf5Results/' + dataSets['measurement'][meas_id]+'.hdf5','r') as f:
            v = f[f'/scan_{scan}/constant_dose/1kGy/v_nm_s'][()]
            dv = f[f'/scan_{scan}/constant_dose/1kGy/v_err'][()]
            
            Gamma = v * q_temp
            tau = np.divide(1,Gamma)
            dtau = np.divide(1,v**2 * q_temp)*dv
            decay_doses[im] = Dosis(tau,ab)
            ddecay_doses[im] = dDosedTau(ab)*dtau
                                   
    ax_d.errorbar(x=fluxes,y=decay_doses,
                yerr=ddecay_doses,
                color=sample_color_dict[str(temp)],
                linestyle='',
                markersize=7,
                marker=marker_dict[str(temp)],
                mec='black',
                alpha=1,
                mew=0.3,
               )

            
            
    ax.errorbar(x=fluxes,y=v1kGy_nms,
                yerr=dv1kGy_nms,
                color=sample_color_dict[str(temp)],
                linestyle='',
                markersize=7,
                marker=marker_dict[str(temp)],
                mec='black',
                alpha=1,
                mew=0.3,
                zorder=10,
               )
    
    y = v1kGy_nms
    if temp < 74:
        v0 = y[0]
    else:
        v0 = y[2]
    popt, pcov = curve_fit(line_fit, fluxes,y,sigma=dv1kGy_nms,
                           p0=[v0,1,]
                          )
    ax.plot(np.logspace(-3,1,50),line_fit(np.logspace(-3,1,50),*popt),color=sample_color_dict[str(temp)],zorder=9)
    ax_i.errorbar(x=popt[0],xerr=np.sqrt(pcov[0,0]),y=popt[1],yerr=np.sqrt(pcov[1,1]),
                  color=sample_color_dict[str(temp)],mec='black',mew=0.3,marker=marker_dict[str(temp)],)
    
    print(popt[1])
    
    v0s[ie]=popt[0]
    dv0s[ie]=np.sqrt(pcov[0,0])
    
    if temp<73:
        phid_soft.append(popt[0]/popt[1])
    elif temp>73:
        phid_strong.append(popt[0]/popt[1])
    
#guides to the eye
fluxes = [flux_dict[str(absorber)] for absorber in [24,18,16,12,8,6,4,2,1]]

ax2.plot([fluxes[0],fluxes[1],fluxes[-1]],[0.5,1.8,1.8],linewidth=2,color='gray',linestyle='dashed',zorder=-1)
ax2.plot([fluxes[0],fluxes[1],fluxes[-1]],[0.2,0.7,0.7],linewidth=2,color='gray',linestyle='dashed',zorder=-1)
ax3.plot([fluxes[0],fluxes[-3],fluxes[-1]],[1,300,300],linewidth=2,color='gray',linestyle='dashed',zorder=-1)
    
    
#rectangles for Phi_D
ax0.add_patch(Rectangle((0.001, 0.08), 0.005, 1700, facecolor ='darkgreen',alpha=0.2))
ax1.add_patch(Rectangle((0.6,1e-3), 1.2,8, facecolor ='darkgreen',alpha=0.2))

ax2.add_patch(Rectangle((0.001, ax2.get_ylim()[0]), 0.005, 20, facecolor ='darkgreen',alpha=0.2))
ax3.add_patch(Rectangle((0.6,ax3.get_ylim()[0]), 1.2,ax3.get_ylim()[1], facecolor ='darkgreen',alpha=0.2))

for a in [ax0,ax1]:
    a.set_yscale('log')
    a.set_xscale('log')
    a.grid('on') 
    a.set_ylabel(r'$\mathrm{v_{1kGy}\:(nm\:s^{-1})}$')
    
ax1.set_xlabel(r'$\Phi$ ($\mathrm{ph}\:\mathrm{s}^{-1}\:\mathrm{nm}^{-2}$)')
ax3.set_xlabel(r'$\Phi$ ($\mathrm{ph}\:\mathrm{s}^{-1}\:\mathrm{nm}^{-2}$)')
ax0.text(0.05,0.9,'a',fontsize=18,transform = ax0.transAxes,color='black',fontweight='bold')
ax1.text(0.05,0.9,'b',fontsize=18,transform = ax1.transAxes,color='black',fontweight='bold')
ax2.text(0.05,0.9,'c',fontsize=18,transform = ax2.transAxes,color='black',fontweight='bold')
ax3.text(0.05,0.9,'d',fontsize=18,transform = ax3.transAxes,color='black',fontweight='bold')

zorder = 2
vlinecol = 'forestgreen'
ax0.axvline(0.003,linestyle=(0,(3,3,1,3)),color=vlinecol,zorder=zorder)
ax2.axvline(0.003,linestyle=(0,(3,3,1,3)),color=vlinecol,zorder=zorder)
ax1.axvline(0.9,linestyle=(0,(3,3,1,3)),color=vlinecol,zorder=zorder)
ax3.axvline(0.9,linestyle=(0,(3,3,1,3)),color=vlinecol,zorder=zorder)

ax0.text(0.006,1000,r'$\Phi_D$',fontsize=10,color=vlinecol)
ax1.text(0.3,3,r'$\Phi_D$',fontsize=10,color=vlinecol)
ax2.text(0.015,0.2,r'$\Phi_D$',fontsize=10,color=vlinecol)
ax3.text(1.3,3,r'$\Phi_D$',fontsize=10,color=vlinecol)

for a in [ax2,ax3]:
    a.set_xscale('log')
    a.set_yscale('log')
    a.grid('on')
    a.set_ylabel(r'$\mathcal{D}_{\mathrm{decay}}\:(\mathrm{kGy})$')

plt.sca(ax2)
lines = ax2.get_lines()
legend0 = plt.legend([lines[i] for i in [0,1,2,3]],
                     ['63','65',
                      '68','70',
                     ],
                     bbox_to_anchor=(0.99,0.01),loc='lower right',
                     title='$\mathrm{T_{prep}(\degree C)}$',
                    framealpha=0.5,ncol=2,columnspacing=0.4)
ax2.add_artist(legend0)

plt.sca(ax3)
lines = ax3.get_lines()
legend1 = plt.legend([lines[i] for i in [0,1,2,3,4]],
                     ['73','75',
                      '78','80','85'
                     ],
                     bbox_to_anchor=(0.99,0.01),loc='lower right',
                     title='$\mathrm{T_{prep}(\degree C)}$',
                    framealpha=0.5,ncol=3,columnspacing=0.4)
ax3.add_artist(legend1)

plt.text(-0.3,0.2,'strong gel networks',rotation='vertical',fontweight='bold',transform = ax1.transAxes)
plt.text(-0.3,0.2,'soft gel networks',rotation='vertical',fontweight='bold',transform = ax0.transAxes)

fig.show()

plt.savefig('Figure6.pdf',dpi=1200,format='pdf')

# Figure 8

In [ ]:
fig5 = plt.figure(constrained_layout=True,figsize=(3.5,1.5))
ax1=plt.axes([0.1,0.3,0.2,0.68])
ax2=plt.axes([0.32,0.3,0.63,0.68])
axbg = plt.axes([0,0,1,1])
axbg.axis('off')
# axbg.xaxis.set_visible(False)
# axbg.yaxis.set_visible(False)


# widths = [1, 3]
# spec5 = fig5.add_gridspec(ncols=2, nrows=1, width_ratios=widths,wspace=0)
# ax1 = fig5.add_subplot(spec5[0])
# ax2 = fig5.add_subplot(spec5[1])

ax1.set_ylim([0,1])
ax2.set_ylim([0,1])
ax1.set_xlim([9e-8,2.3e-6])
ax2.set_xlim([0.008,400])

ax1.yaxis.set_visible(False)
ax2.yaxis.set_visible(False)
for direction in ['left','top','right']:
    ax1.spines[direction].set(visible=False)
    ax2.spines[direction].set(visible=False)

ax1.set_xscale('log')
ax2.set_xscale('log')

ax1.yaxis.tick_left()
ax1.tick_params(labelleft=False)  # don't put tick labels at the top
ax2.yaxis.tick_right()

ax1.plot(3e-7,0.5,marker='d',color='darkgray')
ax2.plot(0.01,0.2,marker='s',color='darkmagenta',alpha=0.5,mew=0)
ax2.add_patch(Rectangle((0.02, 0.5), 0.15, 0.23, facecolor ='darkorange',alpha=0.5))
ax2.add_patch(Rectangle((60, 0.5), 207, 0.43, facecolor ='indianred',alpha=0.5))
ax2.add_patch(Rectangle((0.04, 0.05), 0.4, 0.43, facecolor ='royalblue',alpha=0.5))

ax1.text(3e-7,0.6,'oxide\nglasses',fontsize=8,horizontalalignment='center')
ax2.text(0.07,0.78,'proteins near\nglass transition',fontsize=8,color='darkorange',horizontalalignment='center')
ax2.text(0.5,0.12,'strong\negg white\ngels',fontsize=8,color='royalblue',horizontalalignment='left')
ax2.text(54,0.55,'soft\negg white\ngels',fontsize=8,color='indianred',horizontalalignment='right')

d = .15  # proportion of vertical to horizontal extent of the slanted line
kwargs = dict(marker=[(-0.15, -d), (0.15, d)], markersize=8,
              linestyle="none", color='k', mec='k', mew=1, clip_on=False)
ax1.plot([1], [0], transform=ax1.transAxes, **kwargs)
ax2.plot([0], [0], transform=ax2.transAxes, **kwargs)

axbg.text(0.33,0.48,'hydrated\nlysozyme',horizontalalignment='center',fontsize=8,color='darkmagenta')
axbg.text(0.5,0.05,r'$\alpha(\mathrm{nm^3\:ph})$',horizontalalignment='center')

plt.savefig('Figure8.pdf',format='pdf',dpi=1200)


# Figure S10

In [ ]:
fig, ax = plt.subplots(figsize=(5,7),nrows=10,ncols=5)
plt.subplots_adjust(left=0.2,right=0.98)

#(meas_list_id,q,[series numbers],[col numbers])
ttc_tuples = [(4,0,[1,2,3,4,6],[0,1,2,3,4]),
 (5,0,[1,2,3,4,6],[0,1,2,3,4]),
 (6,0,[1,2,3,4,6],[0,1,2,3,4]),
 (7,0,[1,2,3,4,6],[0,1,2,3,4]),
 (8,0,[1,2,3,4,5],[0,1,2,3,4]),
 (8,5,[1,2,3,4,5],[0,1,2,3,4]),
 (9,5,[1,2,3,4,5],[0,1,2,3,4]),
 (10,5,[1,2,3,4,5],[0,1,2,3,4]),
 (11,5,[1,2,3,4,5],[0,1,2,3,4]),
 (12,5,[1,2,3,4],[0,1,3,4]),
]

for a in ax.flatten():
    a.tick_params(axis='both', which='major', labelsize=5)
    a.set_xticklabels([])


for row, (meas_id, q_ind, series_list, col_list) in enumerate(ttc_tuples):
    with h5py.File('hdf5Results/' + dataSets['measurement'][meas_id]+'.hdf5','r') as f:
        for scan, col in zip(series_list,col_list):
            ttc = f[f'/scan_{scan}/q_{q_ind}/ttc/ttc'][()]
            tt = f[f'/scan_{scan}/q_{q_ind}/ttc/t'][()]
            if row < 5:
                vmin=1
            elif row==5 or row == 6:
                vmin=1.055
            else:
                vmin=1.068
            
            if q_ind ==0:
                vmax = vmin+0.108
            else:
                vmax = vmin+0.097
            #vmin = min(f[f'/scan_{scan}/q_{q_ind}/horizontal_cuts/1_exp_fit/fit_parameter_results'][:,3])
            #vmax = vmin+0.1
            im = ax[row,col].imshow(ttc,origin='lower',extent=([tt[0],tt[-1],tt[0],tt[-1]]),vmin=vmin,vmax=vmax)

ax[0,0].set_title('abs.24',fontsize=7)    
ax[0,1].set_title('abs.18',fontsize=7)   
ax[0,2].set_title('abs.16',fontsize=7)   
ax[0,3].set_title('abs.12',fontsize=7)   
ax[0,4].set_title('abs.8',fontsize=7)   

ax[9,0].set_yticks([0,300])
ax[9,2].axis('off')

for i in range(0,5):
    ax[9,i].set_xlabel('$t$(s)',fontsize=5)

for a,text in zip([ax[0,0],ax[1,0],ax[2,0],ax[3,0],ax[4,0],ax[5,0],ax[6,0],ax[7,0],ax[8,0],ax[9,0]],
                      ['63°C, q=0.006 $\mathrm{nm}^{-1}$','65°C, q=0.006 $\mathrm{nm}^{-1}$','68°C, q=0.006 $\mathrm{nm}^{-1}$','70°C, q=0.006 $\mathrm{nm}^{-1}$','73°C, q=0.006 $\mathrm{nm}^{-1}$',
                      '73°C, q=0.02 $\mathrm{nm}^{-1}$','75°C, q=0.02 $\mathrm{nm}^{-1}$','78°C, q=0.02 $\mathrm{nm}^{-1}$','80°C, q=0.02 $\mathrm{nm}^{-1}$','85°C, q=0.02 $\mathrm{nm}^{-1}$']):
    a.text(-1.7,0.8,text,transform = a.transAxes,color='black',fontsize=5)
    
plt.savefig('FigureS10.pdf',format='pdf',dpi=1200)

# Figure S11

In [ ]:
fig, ax = plt.subplots(figsize=(5,7),nrows=10,ncols=5)
plt.subplots_adjust(left=0.2,right=0.98)

#(meas_list_id,q,[series numbers],[col numbers])
ttc_tuples = [(4,0,[8,10,12,14,16],[0,1,2,3,4]),
 (5,0,[8,10,12,14,16],[0,1,2,3,4]),
 (6,0,[8,10,12,14,16],[0,1,2,3,4]),
 (7,0,[8,10,12,14,16],[0,1,2,3,4]),
 (8,0,[7,9,11,13,15],[0,1,2,3,4]),
 (8,5,[7,9,11,13,15],[0,1,2,3,4]),
 (9,5,[7,9,11,13,15],[0,1,2,3,4]),
 (10,5,[7,9,11,13,15],[0,1,2,3,4]),
 (11,5,[7,9,11,13,15],[0,1,2,3,4]),
 (12,5,[6,8,10],[1,2,4]),
]

for a in ax.flatten():
    a.tick_params(axis='both', which='major', labelsize=5)
    a.set_xticklabels([])


for row, (meas_id, q_ind, series_list, col_list) in enumerate(ttc_tuples):
    with h5py.File('hdf5Results/' + dataSets['measurement'][meas_id]+'.hdf5','r') as f:
        for scan, col in zip(series_list,col_list):
            ttc = f[f'/scan_{scan}/q_{q_ind}/ttc/ttc'][()]
            tt = f[f'/scan_{scan}/q_{q_ind}/ttc/t'][()]
            if row < 5:
                vmin=1
            elif row==5 or row == 6:
                vmin=1.055
            else:
                vmin=1.068
            
            if q_ind ==0:
                vmax = vmin+0.108
            else:
                vmax = vmin+0.097
            #vmin = min(f[f'/scan_{scan}/q_{q_ind}/horizontal_cuts/1_exp_fit/fit_parameter_results'][:,3])
            #vmax = vmin+0.1
            im = ax[row,col].imshow(ttc,origin='lower',extent=([tt[0],tt[-1],tt[0],tt[-1]]),vmin=vmin,vmax=vmax)

ax[0,0].set_title('abs.6',fontsize=7)    
ax[0,1].set_title('abs.4',fontsize=7)   
ax[0,2].set_title('abs.2',fontsize=7)   
ax[0,3].set_title('abs.1',fontsize=7)   
ax[0,4].set_title('abs.0',fontsize=7)   


ax[9,0].axis('off')
ax[9,3].axis('off')
ax[9,3].set_xlim([0,200])
ax[9,3].set_ylim([0,200])

for i in range(0,5):
    ax[9,i].set_xlabel('$t$(s)',fontsize=5)

for a,text in zip([ax[0,0],ax[1,0],ax[2,0],ax[3,0],ax[4,0],ax[5,0],ax[6,0],ax[7,0],ax[8,0]],
                      ['63°C, q=0.006 $\mathrm{nm}^{-1}$','65°C, q=0.006 $\mathrm{nm}^{-1}$','68°C, q=0.006 $\mathrm{nm}^{-1}$','70°C, q=0.006 $\mathrm{nm}^{-1}$','73°C, q=0.006 $\mathrm{nm}^{-1}$',
                      '73°C, q=0.02 $\mathrm{nm}^{-1}$','75°C, q=0.02 $\mathrm{nm}^{-1}$','78°C, q=0.02 $\mathrm{nm}^{-1}$','80°C, q=0.02 $\mathrm{nm}^{-1}$']):
    a.text(-1.7,0.8,text,transform = a.transAxes,color='black',fontsize=5)
    
ax[9,0].text(-1,0.8,'85°C, q=0.02 $\mathrm{nm}^{-1}$',transform = ax[9,0].transAxes,color='black',fontsize=5)

plt.savefig('FigureS11.pdf',format='pdf',dpi=fig.dpi)

# Figure S12

In [ ]:
fig, ax = plt.subplots(ncols=2,nrows=5,sharex=True,sharey=True,figsize=(7,10),squeeze=True)
plt.subplots_adjust(right=0.78,wspace=0,hspace=0,left=0.1,bottom=0.1,top=0.95)
ax0, ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9 = ax[0,0],ax[0,1],ax[1,0],ax[1,1],ax[2,0],ax[2,1],ax[3,0],ax[3,1],ax[4,0],ax[4,1]

colors = plt.cm.turbo_r(np.linspace(1,0,10))
ax_col_flux = fig.add_axes([0.88,0.1,0.02,0.85])
ax_col_doserate = ax_col_flux.twinx()

cmap = ListedColormap([colors[i] for i in range(10)])
norm = mpl.colors.Normalize(vmin=0,vmax=10)
cb = mpl.colorbar.ColorbarBase(ax_col_doserate, cmap=cmap,norm=norm,orientation='vertical')
cb.set_ticks(np.arange(0.5,10.5,1))
cb.set_ticklabels([0.002,0.01,0.03,0.09,0.3,0.6,1,2,3,4])
cb.set_label('$\mathscr{D}\:\mathrm{(kGy\:s^{-1})}$',fontsize=12,labelpad=0)

ax_col_flux.set_ylim([0,10])
ax_col_flux.set_yticks(np.arange(0.5,10.5,1))
ax_col_flux.set_yticklabels([r'3$\times 10^3$',r'2$\times 10^4$',r'4$\times 10^4$',r'1$\times 10^5$',
                             r'5$\times 10^5$',r'9$\times 10^5$',r'2$\times 10^{6}$',r'3$\times 10^{6}$',
                             r'4$\times 10^{6}$',r'6$\times 10^{6}$'])

ax_col_flux.set_ylabel(r'$\Phi$ ($\mathrm{ph}\:\mathrm{s}^{-1}\:\mathrm{\mu m}^{-2}$)')
for a,label,temp_label,q_label in zip([ax0, ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9],
                              ['a','b','c','d','e','f','g','h','i','j'],
                              ['60$\degree$C','63$\degree$C','65$\degree$C','68$\degree$C',
                               '70$\degree$C','73$\degree$C','75$\degree$C','78$\degree$C','80$\degree$C','85$\degree$C'],
                              ['q=0.006$\mathrm{\:nm^{-1}}$','q=0.006$\mathrm{\:nm^{-1}}$','q=0.006$\mathrm{\:nm^{-1}}$',
                               'q=0.006$\mathrm{\:nm^{-1}}$','q=0.006$\mathrm{\:nm^{-1}}$','q=0.006$\mathrm{\:nm^{-1}}$',
                               'q=0.02$\mathrm{\:nm^{-1}}$','q=0.02$\mathrm{\:nm^{-1}}$','q=0.02$\mathrm{\:nm^{-1}}$',
                               'q=0.02$\mathrm{\:nm^{-1}}$']):
    a.set_yscale('log')
    a.set_xscale('log')
    a.set_xlim([0.05,700])
    a.set_ylim([0.0002,10])
    a.grid('on')
    a.text(0.03,0.9,label,fontweight='bold',fontsize=12,transform = a.transAxes,color='black')
    a.text(0.8,0.9,temp_label,fontsize=12,transform = a.transAxes,color='black')
    a.text(0.5,0.05,q_label,fontsize=12,transform = a.transAxes,color='black')

ax0.set_ylabel('$\mathrm{\Gamma\:(s^{-1})}$') 
ax2.set_ylabel('$\mathrm{\Gamma\:(s^{-1})}$')
ax4.set_ylabel('$\mathrm{\Gamma\:(s^{-1})}$')
ax6.set_ylabel('$\mathrm{\Gamma\:(s^{-1})}$')
ax8.set_ylabel('$\mathrm{\Gamma\:(s^{-1})}$')
ax8.set_xlabel('$\mathcal{D}\:\mathrm{(kGy)}$')
ax9.set_xlabel('$\mathcal{D}\:\mathrm{(kGy)}$')


############## 60C ############
sublog = logbook.loc[logbook.temp==60]
result_ids = []
use_entries = [2,4,6,8,10,12]#np.arange(0,11)
exponentiality = [1,1,1,1,1,1]
for ie,(ue,ex) in enumerate(zip(use_entries,exponentiality)):
    res_id = sublog.iloc[ue]['temp'],sublog.iloc[ue]['meas_id'],sublog.iloc[ue]['ser'],sublog.iloc[ue]['ab'],ex
    result_ids.append(res_id)
    
q_ind=0

with h5py.File('hdf5Results/' + sublog.iloc[0]['measurement']+'.hdf5','r') as f:
    combine_Gammas(dataSets,result_ids,q_ind,ax=ax0)

###########63C###############
sublog = logbook.loc[logbook.temp==63]
result_ids = []
use_entries = [0,1,2,3,5,6,7,8,9,10,11,12,13,14,15]
exponentiality = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
for ie,(ue,ex) in enumerate(zip(use_entries,exponentiality)):
    res_id = sublog.iloc[ue]['temp'],sublog.iloc[ue]['meas_id'],sublog.iloc[ue]['ser'],sublog.iloc[ue]['ab'],ex
    result_ids.append(res_id)
    
q_ind=0

with h5py.File('hdf5Results/' + sublog.iloc[0]['measurement']+'.hdf5','r') as f:
    combine_Gammas(dataSets,result_ids,q_ind,ax=ax1)
###########65C###############
sublog = logbook.loc[logbook.temp==65]
result_ids = []
use_entries = [0,1,2,3,5,6,7,8,9,10,11,12,13,14,15]
exponentiality = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
for ie,(ue,ex) in enumerate(zip(use_entries,exponentiality)):
    res_id = sublog.iloc[ue]['temp'],sublog.iloc[ue]['meas_id'],sublog.iloc[ue]['ser'],sublog.iloc[ue]['ab'],ex
    result_ids.append(res_id)
    
q_ind=0

with h5py.File('hdf5Results/' + sublog.iloc[0]['measurement']+'.hdf5','r') as f:
    combine_Gammas(dataSets,result_ids,q_ind,ax=ax2)
    
###########68C###############
sublog = logbook.loc[logbook.temp==68]
result_ids = []
use_entries = [0,1,2,3,5,6,7,8,9,10,11,12,13,14,15]
exponentiality = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
for ie,(ue,ex) in enumerate(zip(use_entries,exponentiality)):
    res_id = sublog.iloc[ue]['temp'],sublog.iloc[ue]['meas_id'],sublog.iloc[ue]['ser'],sublog.iloc[ue]['ab'],ex
    result_ids.append(res_id)
    
q_ind=0

with h5py.File('hdf5Results/' + sublog.iloc[0]['measurement']+'.hdf5','r') as f:
    combine_Gammas(dataSets,result_ids,q_ind,ax=ax3)
    
###########70C###############
sublog = logbook.loc[logbook.temp==70]
result_ids = []
use_entries = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
exponentiality = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
for ie,(ue,ex) in enumerate(zip(use_entries,exponentiality)):
    res_id = sublog.iloc[ue]['temp'],sublog.iloc[ue]['meas_id'],sublog.iloc[ue]['ser'],sublog.iloc[ue]['ab'],ex
    result_ids.append(res_id)
    
q_ind=0

with h5py.File('hdf5Results/' + sublog.iloc[0]['measurement']+'.hdf5','r') as f:
    combine_Gammas(dataSets,result_ids,q_ind,ax=ax4)
    
###########73C###############
sublog = logbook.loc[logbook.temp==73]
result_ids = []
use_entries = [0,1,2,3,4,6,8,10,12,14]
exponentiality = [1,1,1,1,1,1,1,1,1,1]
for ie,(ue,ex) in enumerate(zip(use_entries,exponentiality)):
    res_id = sublog.iloc[ue]['temp'],sublog.iloc[ue]['meas_id'],sublog.iloc[ue]['ser'],sublog.iloc[ue]['ab'],ex
    result_ids.append(res_id)
    
q_ind=0

with h5py.File('hdf5Results/' + sublog.iloc[0]['measurement']+'.hdf5','r') as f:
    combine_Gammas(dataSets,result_ids,q_ind,ax=ax5)

###########75C###############
sublog = logbook.loc[logbook.temp==75]
result_ids = []
use_entries =    [2,3,4,6,8,10,12,14]
exponentiality = [1,1,1,1,2, 2, 2, 2]
for ie,(ue,ex) in enumerate(zip(use_entries,exponentiality)):
    print(sublog.iloc[ue]['ser'])
    res_id = sublog.iloc[ue]['temp'],sublog.iloc[ue]['meas_id'],sublog.iloc[ue]['ser'],sublog.iloc[ue]['ab'],ex
    result_ids.append(res_id)
    
q_ind=5

with h5py.File('hdf5Results/' + sublog.iloc[0]['measurement']+'.hdf5','r') as f:
    combine_Gammas(dataSets,result_ids,q_ind,ax=ax6)
    
###########78C###############
sublog = logbook.loc[logbook.temp==78]
result_ids = []
use_entries =    [2,3,4,8,10,12,14]
exponentiality = [1,1,1,1, 2, 2, 2]
for ie,(ue,ex) in enumerate(zip(use_entries,exponentiality)):
    res_id = sublog.iloc[ue]['temp'],sublog.iloc[ue]['meas_id'],sublog.iloc[ue]['ser'],sublog.iloc[ue]['ab'],ex
    result_ids.append(res_id)
    
q_ind=5

with h5py.File('hdf5Results/' + sublog.iloc[0]['measurement']+'.hdf5','r') as f:
    combine_Gammas(dataSets,result_ids,q_ind,ax=ax7)
    
###########80C###############
sublog = logbook.loc[logbook.temp==80]
result_ids = []
use_entries = [2,3,4,6,8,10,12,14]
exponentiality = [1,1,1,1,1,1,1,1]
for ie,(ue,ex) in enumerate(zip(use_entries,exponentiality)):
    res_id = sublog.iloc[ue]['temp'],sublog.iloc[ue]['meas_id'],sublog.iloc[ue]['ser'],sublog.iloc[ue]['ab'],ex
    result_ids.append(res_id)
    
q_ind=5

with h5py.File('hdf5Results/' + sublog.iloc[0]['measurement']+'.hdf5','r') as f:
    combine_Gammas(dataSets,result_ids,q_ind,ax=ax8)
    
###########85C###############
sublog = logbook.loc[logbook.temp==85]    
result_ids = []
use_entries =    [5,7,9]
exponentiality = [1,1,1]
for ie,(ue,ex) in enumerate(zip(use_entries,exponentiality)):
    res_id = sublog.iloc[ue]['temp'],sublog.iloc[ue]['meas_id'],sublog.iloc[ue]['ser'],sublog.iloc[ue]['ab'],ex
    result_ids.append(res_id)
    
q_ind=5

with h5py.File('hdf5Results/' + sublog.iloc[0]['measurement']+'.hdf5','r') as f:
    combine_Gammas(dataSets,result_ids,q_ind,ax=ax9)
    
plt.savefig('FigureS12.pdf',dpi=1200,format='pdf')

# Figure S13

In [ ]:
fig, (ax0,ax1) = plt.subplots(figsize=(7,3),ncols=2)
plt.subplots_adjust(bottom=0.2)
for a,label in zip([ax0,ax1],['a','b']):
    a.set_xlabel(r'$q\:\mathrm{(nm^{-1})}$')
    a.set_ylabel(r'$\Gamma\:(\mathrm{s^{-1}})$')
    a.text(0.03,0.9,label,fontweight='bold',fontsize=18,transform = a.transAxes,color='black')
    
ax0.set_title('1 kGy',weight='normal')

ax1.set_title('10 kGy',weight='normal')

with h5py.File('hdf5Results/' + dataSets['measurement'][4]+'.hdf5','r') as f:
    scan = 4
    dose = 1
    colorsq = cm.turku(np.linspace(0,1,5))
    q_s = f[f'/scan_{scan}/constant_dose/{dose}kGy/q_nm'][()]
    gamma = f[f'/scan_{scan}/constant_dose/{dose}kGy/fit_parameter_results'][:,1]
    dgamma = f[f'/scan_{scan}/constant_dose/{dose}kGy/fit_parameter_errors'][:,1]
    pars, cov = np.polyfit(x=q_s,y=gamma,deg=1,cov=True)
    ax0.plot(q_s,pars[0]*q_s+pars[1],color='darkgrey',zorder=2)
    for qcut in range(5):
        
        ax0.errorbar(x=q_s[qcut],y=gamma[qcut],yerr=dgamma[qcut],marker='o',
                     linestyle='',zorder=3,color=colorsq[qcut],mew=0.5,mec='black',ecolor='black',capsize=2,elinewidth=0.5)
        

    dose = 10
    q_s = f[f'/scan_{scan}/constant_dose/{dose}kGy/q_nm'][()]
    gamma = f[f'/scan_{scan}/constant_dose/{dose}kGy/fit_parameter_results'][:,1]
    dgamma = f[f'/scan_{scan}/constant_dose/{dose}kGy/fit_parameter_errors'][:,1]
    
    pars, cov = np.polyfit(x=q_s,y=gamma,deg=2,cov=True)
    ax1.plot(q_s,pars[0]*q_s**2+pars[1]*q_s+pars[2],color='darkgrey',zorder=2)
    
    for qcut in range(5):
        
        ax1.errorbar(x=q_s[qcut],y=gamma[qcut],yerr=dgamma[qcut],marker='o',
                     linestyle='',zorder=3,color=colorsq[qcut],mew=0.5,mec='black',ecolor='black',capsize=2,elinewidth=0.5)
        
plt.savefig('FigureS13.pdf',dpi=1200,format='pdf')

# Figure S14

In [ ]:
fig, (ax,ax1,ax2) = plt.subplots(ncols=3,figsize=(7,1.5))
ax.set_xscale('log')
ax.set_ylabel(r'$g^{(2)}(\tau)/\beta -1$')
ax.set_xlabel(r'$\tau$')
ax.text(0.87,0.87,'a',fontsize=12,transform = ax.transAxes, fontweight='bold')

ax1.set_xscale('log')
ax1.set_xlabel('$q\delta$')
ax1.set_ylabel('$k$')
ax1.text(0.87,0.87,'b',fontsize=12,transform = ax1.transAxes, fontweight='bold')


ax2.set_ylabel(r'$\Gamma/(\gamma q\delta)$')
ax2.set_xscale('log')
ax2.set_xlabel('$q\delta$')
ax2.text(0.87,0.87,'c',fontsize=12,transform = ax2.transAxes, fontweight='bold')

plt.subplots_adjust(bottom=0.25,wspace=0.4,left=0.15,top=0.95,right=0.82)

ax_col = fig.add_axes([0.83,0.25,0.02,0.7])

#model
N_steps=200 # calculate sum of f(q) for up to N_steps events
gamma=120 # rate of microscopic relaxation events

num_t = 60 # number of points in time
time_array =np.logspace(-2, 0, num=num_t) # time points at which to calculate f(q,t)

colors = plt.cm.PuOr(np.linspace(0,1,12))

for s,sigma_q in enumerate(np.logspace(-2,0,12)):

    g2 = np.zeros(shape=(num_t))
    for tt,time in enumerate(time_array):#tt: number of time step, time(s)

        intermediate_scat_func = 0

        for N in range(0, N_steps):
            P_tN = poisson.pmf(N,gamma*time) # Probability of N events in time "time"
            h_qN = np.exp(-(sigma_q*N)**2)  # degree of decorrelation of each step 
                                            # assuming ballistic motion and gaussian distribution of steps
            intermediate_scat_func = intermediate_scat_func+h_qN*P_tN

        g2[tt]=(intermediate_scat_func**2)

    ax.plot(time_array,g2,marker='s',linestyle='',color=colors[s],markersize=3)

    #fit
    def g2_model(t,Gamma,KWW):
        return(np.exp(-2*(Gamma*t)**KWW))
    popt, pcov = curve_fit(g2_model, time_array, g2, p0=(sigma_q*gamma,2))
    ax.plot(time_array,g2_model(time_array,*popt),color=colors[s],markersize=3)
    print(popt[0]/(gamma*sigma_q))
    ax1.errorbar(x=sigma_q,y=popt[1],yerr=np.sqrt(pcov[1,1]),marker='s',color=colors[s],markersize=3)
    ax2.errorbar(x=sigma_q,y=popt[0]/(gamma*sigma_q),yerr=np.sqrt(pcov[0,0])/(gamma*sigma_q),marker='s',color=colors[s],markersize=3)
    
#colorbar
cmap = plt.cm.PuOr
norm = mpl.colors.LogNorm(vmin=0.02, vmax=1)
cb1 = mpl.colorbar.ColorbarBase(ax_col, cmap=cmap,
                                norm=norm,
                                orientation='vertical')
cb1.set_label('$q\delta$')

ax2.axhline(1,color='grey',zorder=-1)

plt.savefig('FigureS14.pdf',dpi=1200,format='pdf')

# Figure S15

In [ ]:
names = ['Ruta et al. \n oxide glasses', 'Bin et al. \n hydr. Lysozyme','Chushkin et al.\n proteins almost glass','Chushkin et al.\n protein glass', 'Egg white 80$\degree$C', 'Akhundzadeh \n IgG PEG LLPS', 'Egg white 73$\degree$C','Egg white 68$\degree$C','Egg white 63$\degree$C' ]
alphas = [3e-7,0.01,0.02,0.067,0.038,0.15,0.4,207,60]

In [ ]:
# Choose some nice levels
levels=[-0.5,0.5,-0.5,2.5,-3,4,-4,-0.5,0.5]

# Create figure and plot a stem plot with the date
fig, ax = plt.subplots(figsize=(8.8, 4), constrained_layout=True)
#ax.set(title=r'\alpha for different systems investigated with XPCS')

ax.vlines(alphas, 0, levels, color="tab:red")  # The vertical stems.
ax.plot(alphas, np.zeros_like(alphas), "-o",
        color="k", markerfacecolor="w")  # Baseline and markers on it.

# annotate lines
for d, l, r in zip(alphas, levels, names):
    ax.annotate(r, xy=(d, l),
                fontsize=12,
                xytext=(-3, np.sign(l)*3), textcoords="offset points",
                horizontalalignment="right",
                verticalalignment="bottom" if l > 0 else "top")


ax.set_xscale('log')
ax.set_xlabel(r"""$\alpha(\mathrm{nm^3\:ph^{-1}})$""",fontsize=15)
# remove y axis and spines
ax.yaxis.set_visible(False)
for direction in ['left','top','right']:
    ax.spines[direction].set(visible=False)
    
ax.set_xticks([1e-7,1e-6,1e-5,1e-4,1e-3,1e-2,1e-1,1e-0,1e1,1e2])
ax.set_xticklabels(['$10^{-7}$','$10^{-6}$','$10^{-5}$','$10^{-4}$','$10^{-3}$','$10^{-2}$','$10^{-1}$','$10^{0}$','$10^{1}$','$10^{2}$'],fontsize=14)

ax.margins(y=0.1)
plt.show()

#plt.savefig('Figure_Alphas.png',format='png',dpi=400,transparent=True)
plt.savefig('FigureS15.pdf',format='pdf',dpi=1200)

# Figure S16

In [ ]:
fig, axen = plt.subplots(nrows = 6, ncols = 7,figsize=(7,5.68),sharex=True,sharey=True)
plt.subplots_adjust(wspace=0,hspace=0,left=0.2,top=0.85)
ax = axen.flatten()
axes0 = []
axes1 = []
for a in [0,1,2,7,8,9,14,15,16,21,22,23,28,29,30,35,36,37]:
    axes0.append(ax[a])
    axes1.append(ax[a+4])
for a in [0,7,14,21,28,35]:
    ax[a+3].axis('off')

with h5py.File('hdf5Results/m042_eggwhite_egg1_50mMNaCl_cooked70_bd1.hdf5','r') as f:
    for ia,(a, scan) in enumerate(zip(axes0, [1,2,3,4,5,6,9,10,11,12,13,14,15,16,17,18,19,20])):
        if ia ==5:
            continue
        tt = f[f'/scan_{scan}/q_0/ttc/t'][()]
        ttc = f[f'/scan_{scan}/q_0/ttc/ttc'][()]
        im = a.imshow(ttc,origin='lower',extent=([tt[0],tt[-1],tt[0],tt[-1]]))
        
with h5py.File('hdf5Results/m044_eggwhite_egg1_50mMNaCl_cooked80_bd.hdf5','r') as f:
    for ia,(a, scan) in enumerate(zip(axes1, [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18])):
        tt = f[f'/scan_{scan}/q_5/ttc/t'][()]
        ttc = f[f'/scan_{scan}/q_5/ttc/ttc'][()]
        im = a.imshow(ttc,origin='lower',extent=([tt[0],tt[-1],tt[0],tt[-1]]))
for a in ax[range(0,36,7)]:
    a.set_ylabel('$t_2$(s)')
    a.set_yticks([0,100])
    
for a in [ax[35],ax[36],ax[37],ax[39],ax[40],ax[41]]:
    a.set_xlabel('$t_1$(s)')
    
for a,doserate in zip([ax[0],ax[7],ax[14],ax[21],ax[28],ax[35]],[0.03,0.09,0.3,1.1,2.1,4]):
    a.text(-2,0.8,'$\mathscr{D}=$'+f'{doserate}'+'$\mathrm{\:kGy\:s^{-1}}$',transform = a.transAxes,color='black')
ax[0].set_title('scan 1')
ax[1].set_title('scan 2')
ax[2].set_title('scan 3')

ax[4].set_title('scan 1')
ax[5].set_title('scan 2')
ax[6].set_title('scan 3')

ax[1].text(-0.5,1.6,'soft gel network $70\degree$C\n q=0.006 nm$^{-1}$',transform = ax[1].transAxes)
ax[5].text(-.5,1.6,'strong gel network $80\degree$C\n q=0.02 nm$^{-1}$',transform = ax[5].transAxes)
ax[9].text(0.1,0.4,'no beam',transform = ax[9].transAxes)

plt.savefig('FigureS16.pdf',format='pdf',dpi=1200)